In [2]:

import torch
import numpy as np
import cv2
from segment_anything import sam_model_registry, SamPredictor
from ultralytics import YOLO



In [3]:

def highlight_people_cars_and_bikes(full_path_input_image, 
                                  color_scale_image, 
                                  color_scale_people, 
                                  color_scale_cars, 
                                  color_scale_bikes, 
                                  full_path_output_image):
    # Cargar modelos
    yolo_model = YOLO('yolov8x.pt')
    sam = sam_model_registry["default"]("sam_vit_h_4b8939.pth")
    sam_predictor = SamPredictor(sam)
    
    # Cargar y convertir imagen a escala de grises
    image = cv2.imread(full_path_input_image)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    colored_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
    
    # Convertir imagen base al color_scale_image
    colored_image = cv2.multiply(colored_image, 
                               np.array(color_scale_image)/255.0)
    
    # Detectar objetos con YOLO
    results = yolo_model(image)
    
    # Preparar imagen para SAM
    sam_predictor.set_image(image)
    
    for result in results[0]:
        bbox = result.boxes.xyxy[0].cpu().numpy()
        class_id = int(result.boxes.cls[0].item())
        conf = result.boxes.conf[0].item()
        
        if conf < 0.5:  # Umbral de confianza
            continue
            
        # Convertir bbox a formato input_box para SAM
        input_box = np.array([bbox[0], bbox[1], bbox[2], bbox[3]])
        
        # Obtener máscara de SAM
        masks, _, _ = sam_predictor.predict(
            box=input_box,
            multimask_output=False
        )
        mask = masks[0]
        
        # Aplicar color según la clase
        if class_id == 0:  # persona
            color = np.array(color_scale_people)/255.0
        elif class_id == 2:  # coche
            color = np.array(color_scale_cars)/255.0
        elif class_id == 1:  # bicicleta/moto
            color = np.array(color_scale_bikes)/255.0
        else:
            continue
            
        # Aplicar color solo en los píxeles de la máscara
        colored_image[mask] = image[mask] * color
    
    # Guardar imagen resultante
    cv2.imwrite(full_path_output_image, colored_image)

In [4]:
# Ejemplo de colores en formato BGR
highlight_people_cars_and_bikes(
    'p1.jpg',
    color_scale_image=[0.5, 0.5, 0.5],  # Gris
    color_scale_people=[0, 0, 1],        # Rojo
    color_scale_cars=[0, 1, 0],          # Verde
    color_scale_bikes=[1, 0, 0],         # Azul
    full_path_output_image = 'output.jpg'
)

: 